## Trexquant Hangman Game Solver

This project was developed as part of the Trexquant interview challenge. The task was to build an algorithm that plays Hangman via a remote API, using only a restricted training dictionary of 250,000 words.

To significantly outperform a basic frequency-based benchmark (18% success rate), I designed a deep learning model with an attention-based bidirectional LSTM architecture. The model predicts likely letters based on partially masked words, vowel ratio, and normalized word length.

It was trained on synthetically generated masked examples and integrated into a fully automated guessing agent that interacts with Trexquant’s API to play and evaluate games in real-time.


## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA version in PyTorch:", torch.version.cuda)
print("Is CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


PyTorch version: 2.5.0+cu118
CUDA version in PyTorch: 11.8
Is CUDA available: True
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU


In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import numpy as np

# Disable warnings
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [2]:
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)


In [3]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter(
            "".join(self.full_dictionary)
        ).most_common()

        self.current_dictionary = []

        # Initialize character mappings
        self.char_to_idx = {c: i + 1 for i, c in enumerate("abcdefghijklmnopqrstuvwxyz")}
        self.char_to_idx["_"] = 27
        self.idx_to_char = {i: c for c, i in self.char_to_idx.items()}
        self.vocab_size = len(self.char_to_idx) + 1  # For padding index 0
        self.max_len = max(len(word) for word in self.full_dictionary)

       # Set device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print('Using device:', self.device)

        # Load the model
        embedding_dim = 64
        hidden_dim = 128
        output_dim = 26
        num_layers = 4
        self.model = HangmanModel(
        self.vocab_size, embedding_dim, hidden_dim, output_dim, num_layers=num_layers).to(self.device)
        self.model.load_state_dict(torch.load("hangman_model_ATTENTION.pth", map_location=self.device))
        self.model.eval()

    @staticmethod
    def determine_hangman_url():
        links = ["https://trexsim.com", "https://sg.trexsim.com"]

        data = {link: 0 for link in links}

        for link in links:
            try:
                requests.get(link)
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s
            except:
                data[link] = float('inf')

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += "/trexsim/hangman"
        return link

    def guess(self, word):
        # Prepare the input
        word = word.strip().replace(" ", "")
        input_seq = []
        known_letters = []
        for c in word:
            if c == "_":
                input_seq.append(self.char_to_idx["_"])
            else:
                input_seq.append(self.char_to_idx[c])
                known_letters.append(c)
        # Pad input_seq to max_len
        input_seq = input_seq + [0] * (self.max_len - len(input_seq))
        input_tensor = torch.tensor([input_seq]).to(self.device)

        # Calculate the current vowel ratio
        vowels = set('aeiou')
        total_known_letters = len(known_letters)
        vowel_count = sum(1 for c in known_letters if c in vowels)
        if total_known_letters > 0:
            vowel_ratio = vowel_count / total_known_letters
        else:
            vowel_ratio = 0.0
        vowel_ratio_tensor = torch.tensor([[vowel_ratio]], dtype=torch.float).to(self.device)

        # Calculate the word length
        word_length = len(word) / self.max_len  # Normalize word length
        word_length_tensor = torch.tensor([[word_length]], dtype=torch.float).to(self.device)

        # Run the model
        with torch.no_grad():
            output = self.model(input_tensor, vowel_ratio_tensor, word_length_tensor)
        output = output.squeeze(0).cpu().numpy()

        # Get probabilities for letters not yet guessed
        letters_not_guessed = [
            c for c in "abcdefghijklmnopqrstuvwxyz" if c not in self.guessed_letters
        ]

        # Implement the logic
        word_length = len(word)
        if word_length > 0:
            vowel_ratio_in_word = vowel_count / word_length
            if vowel_ratio_in_word >= 0.55:
                # Only guess consonants from now on
                letters_not_guessed = [c for c in letters_not_guessed if c not in vowels]
        else:
            vowel_ratio_in_word = 0.0

        probs_not_guessed = {
            c: output[self.char_to_idx[c] - 1] for c in letters_not_guessed
        }

        # Select the letter with the highest probability
        guess_letter = max(probs_not_guessed, key=probs_not_guessed.get)
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location, "r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # Reset guessed letters and current dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice": practice})
        if response.get("status") == "approved":
            game_id = response.get("game_id")
            word = response.get("word")
            tries_remains = response.get("tries_remains")
            if verbose:
                print(
                    "Successfully started a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(
                        game_id, tries_remains, word
                    )
                )
            while tries_remains > 0:
                # Get guessed letter from user code
                guess_letter = self.guess(word)

                # Append guessed letter to guessed letters list
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request(
                        "/guess_letter",
                        {
                            "request": "guess_letter",
                            "game_id": game_id,
                            "letter": guess_letter,
                        },
                    )
                except HangmanAPIError:
                    print("HangmanAPIError exception caught on request.")
                    continue
                except Exception as e:
                    print("Other exception caught on request.")
                    raise e

                if verbose:
                    print("Server response: {0}".format(res))
                status = res.get("status")
                tries_remains = res.get("tries_remains")
                if status == "success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status == "failed":
                    reason = res.get("reason", "# of tries exceeded!")
                    if verbose:
                        print("Failed game: {0}. Reason: {1}".format(game_id, reason))
                    return False
                elif status == "ongoing":
                    word = res.get("word")
        else:
            if verbose:
                print("Failed to start a new game")
        return False

    def my_status(self):
        return self.request("/my_status", {})

    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been included.
        if self.access_token:
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False,
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)
            except requests.exceptions.ConnectionError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep

                )

        headers = response.headers
        if "json" in headers.get("content-type", ""):
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError("Maintype was not text, or querystring")

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

In [4]:
import torch.nn.functional as F

class HangmanModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers=4, dropout=0.1):
        super(HangmanModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True
        )
        # Attention layer
        self.attention = nn.Linear(hidden_dim * 2, 1)
        
        self.fc = nn.Linear(hidden_dim * 2 + 1 + 1, output_dim)  # +1 for vowel_ratio, +1 for word_length
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, vowel_ratio, word_length):
        embeds = self.embedding(x) 
        embeds = self.dropout(embeds)
        
        outputs, (hn, cn) = self.lstm(embeds) 
        
        # Compute attention scores
        attn_energies = self.attention(outputs)
        attn_energies = attn_energies.squeeze(2)
        
        # Compute attention weights
        attn_weights = F.softmax(attn_energies, dim=1) 
        
        # Compute context vector
        context = torch.bmm(attn_weights.unsqueeze(1), outputs) 
        context = context.squeeze(1)
        
        # Concatenate context with vowel_ratio and word_length
        extra_features = torch.cat((vowel_ratio, word_length), dim=1)
        combined = torch.cat((context, extra_features), dim=1)
        combined = self.dropout(combined)
        
        # Final output
        out = self.fc(combined)
        out = self.sigmoid(out)
        return out

In [5]:
class HangmanDataset(Dataset):
    def __init__(self, words, char_to_idx, max_len):
        self.words = words
        self.data = []
        self.char_to_idx = char_to_idx
        self.max_len = max_len
        vowels = set('aeiou')
        
        for word in words:
            word = word.lower()
            word_indices = [char_to_idx[c] for c in word]
            for _ in range(25):  # Generate masked versions per word
                masked_word_indices = word_indices.copy()
                # Randomly mask out some letters
                if len(word) > 1:
                    num_mask = random.randint(1, len(word) - 1)
                else:
                    num_mask = 1
                mask_indices = random.sample(range(len(word)), num_mask)
                known_letters = []
                for idx in range(len(word)):
                    if idx in mask_indices:
                        masked_word_indices[idx] = char_to_idx["_"]
                    else:
                        known_letters.append(word[idx])
                # Prepare input and output
                word_length = len(word)/self.max_len  # Calculate the length of the word
                input_seq = masked_word_indices
                output_vec = [0] * 26
                for c in word:
                    output_vec[char_to_idx[c] - 1] = 1  # letters 'a'-'z' map to indices 1-26
                # Calculate vowel ratio
                total_known_letters = len(known_letters)
                if total_known_letters > 0:
                    vowel_count = sum(1 for c in known_letters if c in vowels)
                    vowel_ratio = vowel_count / total_known_letters
                else:
                    vowel_ratio = 0.0
                # Pad input_seq
                input_seq = input_seq + [0] * (self.max_len - len(input_seq))
                self.data.append((input_seq, vowel_ratio, word_length, output_vec))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_seq, vowel_ratio, word_length, output_vec = self.data[idx]
        return (
            torch.tensor(input_seq),
            torch.tensor([vowel_ratio], dtype=torch.float),
            torch.tensor([word_length], dtype=torch.float),
            torch.tensor(output_vec, dtype=torch.float),
    )

In [6]:
def train_model():
    # Initialize character mappings
    char_to_idx = {c: i + 1 for i, c in enumerate("abcdefghijklmnopqrstuvwxyz")}
    char_to_idx["_"] = 27
    idx_to_char = {i: c for c, i in char_to_idx.items()}
    vocab_size = len(char_to_idx) + 1  # For padding index 0

    # Load the full dictionary
    with open("words_250000_train.txt", "r") as f:
        training_words = f.read().splitlines()

    max_len = max(len(word) for word in training_words)

    dataset = HangmanDataset(training_words, char_to_idx, max_len)
    dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=0)

    # Define the model
    embedding_dim = 64
    hidden_dim = 128
    output_dim = 26
    num_epochs = 30

    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)

    model = HangmanModel(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

    # learning rate scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    # Train the model
    for epoch in range(num_epochs):
        start_time = time.time()
        total_loss = 0
        model.train()
        for inputs, vowel_ratios, word_lengths, targets in dataloader:
            inputs = inputs.to(device)
            vowel_ratios = vowel_ratios.to(device)
            word_lengths = word_lengths.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs, vowel_ratios, word_lengths)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        epoch_time = time.time() - start_time

        # Update the scheduler
        scheduler.step()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Time: {epoch_time:.2f} seconds")

    # Save the model
    torch.save(model.state_dict(), "hangman_model_ATTENTION.pth")


In [7]:
train_model()

Using device: cuda
Epoch 1/30, Loss: 0.2630, Time: 589.69 seconds
Epoch 2/30, Loss: 0.2417, Time: 584.31 seconds
Epoch 3/30, Loss: 0.2372, Time: 584.28 seconds
Epoch 4/30, Loss: 0.2346, Time: 582.64 seconds
Epoch 5/30, Loss: 0.2329, Time: 577.75 seconds
Epoch 6/30, Loss: 0.2317, Time: 599.42 seconds
Epoch 7/30, Loss: 0.2308, Time: 610.01 seconds
Epoch 8/30, Loss: 0.2301, Time: 610.26 seconds
Epoch 9/30, Loss: 0.2295, Time: 614.63 seconds
Epoch 10/30, Loss: 0.2290, Time: 607.74 seconds
Epoch 11/30, Loss: 0.2255, Time: 613.44 seconds
Epoch 12/30, Loss: 0.2247, Time: 611.70 seconds
Epoch 13/30, Loss: 0.2244, Time: 610.31 seconds
Epoch 14/30, Loss: 0.2241, Time: 824.86 seconds
Epoch 15/30, Loss: 0.2239, Time: 864.33 seconds
Epoch 16/30, Loss: 0.2237, Time: 868.47 seconds
Epoch 17/30, Loss: 0.2235, Time: 678.81 seconds
Epoch 18/30, Loss: 0.2234, Time: 578.15 seconds
Epoch 19/30, Loss: 0.2232, Time: 572.24 seconds
Epoch 20/30, Loss: 0.2231, Time: 576.29 seconds
Epoch 21/30, Loss: 0.2206, Tim

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [7]:
api = HangmanAPI(access_token="fc4186ffea9ce6b4b7ff4fdc7e4e61", timeout=2000)


Using device: cuda


C:\Users\stoic\AppData\Local\Temp\ipykernel_19256\3990892376.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load("hangman_model_ATTENT

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [15]:
for i in range(1000):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully started a new game! Game ID: 1ff22fae4b2e. # of tries remaining: 6. Word: _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '1ff22fae4b2e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ '}
Guessing letter: a
Server response: {'game_id': '1ff22fae4b2e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ a _ _ a _ '}
Guessing letter: n
Server response: {'game_id': '1ff22fae4b2e', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ a _ _ a _ '}
Guessing letter: l
Server response: {'game_id': '1ff22fae4b2e', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ a _ _ a _ '}
Guessing letter: r
Server response: {'game_id': '1ff22fae4b2e', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ a _ _ a _ '}
Guessing letter: m
Server response: {'game_id': '1ff22fae4b2e', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ a m _ a _ '}
Guessing letter: b
Server response: {'game_id': '1ff22fae4b2e', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ a m b a _

run 42167 practice games out of an allotted 100,000. practice success rate so far = 0.570
Successfully started a new game! Game ID: 329e367039d4. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '329e367039d4', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ _ _ _ _ _ _ e _ '}
Guessing letter: d
Server response: {'game_id': '329e367039d4', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ _ _ _ _ _ _ _ e _ '}
Guessing letter: r
Server response: {'game_id': '329e367039d4', 'status': 'ongoing', 'tries_remains': 5, 'word': 'r e _ _ _ _ _ _ _ _ _ e _ '}
Guessing letter: t
Server response: {'game_id': '329e367039d4', 'status': 'ongoing', 'tries_remains': 5, 'word': 'r e _ _ _ _ t _ _ _ t e _ '}
Guessing letter: a
Server response: {'game_id': '329e367039d4', 'status': 'ongoing', 'tries_remains': 5, 'word': 'r e _ a _ _ t _ _ a t e _ '}
Guessing letter: i
Server response: {'game_id': '329e367039d4', 'status': 'ongoi

run 42172 practice games out of an allotted 100,000. practice success rate so far = 0.570
Successfully started a new game! Game ID: 01d506344850. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '01d506344850', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Server response: {'game_id': '01d506344850', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ i _ _ '}
Guessing letter: n
Server response: {'game_id': '01d506344850', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ _ _ _ _ i _ _ '}
Guessing letter: a
Server response: {'game_id': '01d506344850', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ a _ _ a _ _ _ _ i _ _ '}
Guessing letter: t
Server response: {'game_id': '01d506344850', 'status': 'ongoing', 'tries_remains': 4, 'word': 't _ a _ _ a t _ _ _ i _ _ '}
Guessing letter: r
Server response: {'game_id': '01d506344850', 'status': 'ongoi

Server response: {'game_id': '6d1c7a6e691b', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ i _ i g r e i n g '}
Guessing letter: m
Server response: {'game_id': '6d1c7a6e691b', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ i _ i g r e i n g '}
Guessing letter: f
Server response: {'game_id': '6d1c7a6e691b', 'status': 'ongoing', 'tries_remains': 2, 'word': 'f i _ i g r e i n g '}
Guessing letter: l
Server response: {'game_id': '6d1c7a6e691b', 'status': 'success', 'tries_remains': 2, 'word': 'f i l i g r e i n g '}
Successfully finished game: 6d1c7a6e691b
run 42177 practice games out of an allotted 100,000. practice success rate so far = 0.570
Successfully started a new game! Game ID: 86297ee58f67. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': '86297ee58f67', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ e _ _ '}
Guessing letter: s
Server response: {'game_id': '86297ee58f67', 'status': 'ongoing', 'tries_remains': 6, '

Server response: {'game_id': '6add58cdae54', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ i m p l e '}
Guessing letter: t
Server response: {'game_id': '6add58cdae54', 'status': 'failed', 'tries_remains': 0, 'word': '_ i m p l e '}
Failed game: 6add58cdae54. Reason: # of tries exceeded!
run 42182 practice games out of an allotted 100,000. practice success rate so far = 0.570
Successfully started a new game! Game ID: ef7709ac137f. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'ef7709ac137f', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ _ _ '}
Guessing letter: r
Server response: {'game_id': 'ef7709ac137f', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ _ _ _ '}
Guessing letter: s
Server response: {'game_id': 'ef7709ac137f', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ s _ _ '}
Guessing letter: i
Server response: {'game_id': 'ef7709ac137f', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e _ 

...

Server response: {'game_id': '7f4008cac8fe', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ a r _ _ _ _ _ e r '}
Guessing letter: s
Server response: {'game_id': '7f4008cac8fe', 'status': 'ongoing', 'tries_remains': 4, 'word': 's _ a r _ s _ _ _ e r '}
Guessing letter: h
Server response: {'game_id': '7f4008cac8fe', 'status': 'ongoing', 'tries_remains': 4, 'word': 's h a r _ s _ _ _ e r '}
Guessing letter: p
Server response: {'game_id': '7f4008cac8fe', 'status': 'ongoing', 'tries_remains': 3, 'word': 's h a r _ s _ _ _ e r '}
Guessing letter: k
Server response: {'game_id': '7f4008cac8fe', 'status': 'ongoing', 'tries_remains': 3, 'word': 's h a r k s _ _ k e r '}
Guessing letter: m
Server response: {'game_id': '7f4008cac8fe', 'status': 'ongoing', 'tries_remains': 2, 'word': 's h a r k s _ _ k e r '}
Guessing letter: u
Server response: {'game_id': '7f4008cac8fe', 'status': 'ongoing', 'tries_remains': 2, 'word': 's h a r k s u _ k e r '}
Guessing letter: c
Server response: {'game_id'

Server response: {'game_id': '5646f2e5ef21', 'status': 'ongoing', 'tries_remains': 2, 'word': 'd _ s a _ t a t i o n '}
Guessing letter: u
Server response: {'game_id': '5646f2e5ef21', 'status': 'ongoing', 'tries_remains': 1, 'word': 'd _ s a _ t a t i o n '}
Guessing letter: p
Server response: {'game_id': '5646f2e5ef21', 'status': 'ongoing', 'tries_remains': 1, 'word': 'd _ s a p t a t i o n '}
Guessing letter: m
Server response: {'game_id': '5646f2e5ef21', 'status': 'failed', 'tries_remains': 0, 'word': 'd _ s a p t a t i o n '}
Failed game: 5646f2e5ef21. Reason: # of tries exceeded!
run 43145 practice games out of an allotted 100,000. practice success rate so far = 0.571
Successfully started a new game! Game ID: f3791e4f99ba. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'f3791e4f99ba', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ '}
Guessing letter: a
Server response: {'game_id': 'f3791e4f99ba', 'status': 'ongoing',

run 43150 practice games out of an allotted 100,000. practice success rate so far = 0.571
Successfully started a new game! Game ID: f0b576d60232. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Server response: {'game_id': 'f0b576d60232', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ '}
Guessing letter: a
Server response: {'game_id': 'f0b576d60232', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ a _ '}
Guessing letter: n
Server response: {'game_id': 'f0b576d60232', 'status': 'ongoing', 'tries_remains': 5, 'word': 'n _ _ _ _ a _ '}
Guessing letter: l
Server response: {'game_id': 'f0b576d60232', 'status': 'ongoing', 'tries_remains': 5, 'word': 'n _ _ _ _ a l '}
Guessing letter: i
Server response: {'game_id': 'f0b576d60232', 'status': 'ongoing', 'tries_remains': 5, 'word': 'n _ _ i _ a l '}
Guessing letter: o
Server response: {'game_id': 'f0b576d60232', 'status': 'ongoing', 'tries_remains': 5, 'word': 'n o _ i _ a l '}
Guessing letter: c
Se

Server response: {'game_id': 'd66e28c181ab', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ _ l e s s '}
Guessing letter: t
Server response: {'game_id': 'd66e28c181ab', 'status': 'ongoing', 'tries_remains': 4, 'word': 'a _ _ l e s s '}
Guessing letter: m
Server response: {'game_id': 'd66e28c181ab', 'status': 'ongoing', 'tries_remains': 3, 'word': 'a _ _ l e s s '}
Guessing letter: i
Server response: {'game_id': 'd66e28c181ab', 'status': 'ongoing', 'tries_remains': 2, 'word': 'a _ _ l e s s '}
Guessing letter: c
Server response: {'game_id': 'd66e28c181ab', 'status': 'ongoing', 'tries_remains': 1, 'word': 'a _ _ l e s s '}
Guessing letter: d
Server response: {'game_id': 'd66e28c181ab', 'status': 'failed', 'tries_remains': 0, 'word': 'a _ _ l e s s '}
Failed game: d66e28c181ab. Reason: # of tries exceeded!
run 43155 practice games out of an allotted 100,000. practice success rate so far = 0.571
Successfully started a new game! Game ID: 1b45d5464083. # of tries remaining: 6. Word: _

Server response: {'game_id': '80da446231a6', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ a _ _ _ r '}
Guessing letter: s
Server response: {'game_id': '80da446231a6', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ a _ _ _ r '}
Guessing letter: t
Server response: {'game_id': '80da446231a6', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ a _ _ _ r '}
Guessing letter: i
Server response: {'game_id': '80da446231a6', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ a _ _ _ r '}
Guessing letter: n
Server response: {'game_id': '80da446231a6', 'status': 'ongoing', 'tries_remains': 1, 'word': 'n a _ _ _ r '}
Guessing letter: u
Server response: {'game_id': '80da446231a6', 'status': 'ongoing', 'tries_remains': 1, 'word': 'n a _ _ u r '}
Guessing letter: o
Server response: {'game_id': '80da446231a6', 'status': 'failed', 'tries_remains': 0, 'word': 'n a _ _ u r '}
Failed game: 80da446231a6. Reason: # of tries exceeded!
run 43160 practice games out of an allotted 100,000. practice

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [8]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

Playing  378  th game
Playing  379  th game
Playing  380  th game
Playing  381  th game
Playing  382  th game
Playing  383  th game
Playing  384  th game
Playing  385  th game
Playing  386  th game
Playing  387  th game
Playing  388  th game
Playing  389  th game
Playing  390  th game
Playing  391  th game
Playing  392  th game
Playing  393  th game
Playing  394  th game
Playing  395  th game
Playing  396  th game
Playing  397  th game
Playing  398  th game
Playing  399  th game
Playing  400  th game
Playing  401  th game
Playing  402  th game
Playing  403  th game
Playing  404  th game
Playing  405  th game
Playing  406  th game
Playing  407  th game
Playing  408  th game
Playing  409  th game
Playing  410  th game
Playing  411  th game
Playing  412  th game
Playing  413  th game
Playing  414  th game
Playing  415  th game
Playing  416  th game
Playing  417  th game
Playing  418  th game
Playing  419  th game
Playing  420  th game
Playing  421  th game
Playing  422  th game
Playing  4

Playing  751  th game
Playing  752  th game
Playing  753  th game
Playing  754  th game
Playing  755  th game
Playing  756  th game
Playing  757  th game
Playing  758  th game
Playing  759  th game
Playing  760  th game
Playing  761  th game
Playing  762  th game
Playing  763  th game
Playing  764  th game
Playing  765  th game
Playing  766  th game
Playing  767  th game
Playing  768  th game
Playing  769  th game
Playing  770  th game
Playing  771  th game
Playing  772  th game
Playing  773  th game
Playing  774  th game
Playing  775  th game
Playing  776  th game
Playing  777  th game
Playing  778  th game
Playing  779  th game
Playing  780  th game
Playing  781  th game
Playing  782  th game
Playing  783  th game
Playing  784  th game
Playing  785  th game
Playing  786  th game
Playing  787  th game
Playing  788  th game
Playing  789  th game
Playing  790  th game
Playing  791  th game
Playing  792  th game
Playing  793  th game
Playing  794  th game
Playing  795  th game
Playing  7

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [9]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.604
